# Installing Required Packages

### Note: Resolving `keras.src` Namespace Issue
When using TensorFlow and TensorFlow Model Optimization in Colab, you may encounter a `keras.src` namespace issue, causing incompatibility with `tensorflow_model_optimization.quantization.keras`. To resolve this:

1. Set the `KERAS_BACKEND` environment variable to `tensorflow` before importing TensorFlow.
2. Ensure you are using compatible versions of TensorFlow (`>=2.12`) and TensorFlow Model Optimization.
3. Clone the model using `tensorflow.keras.models.clone_model()` to ensure it aligns with the `tensorflow.keras` namespace.
4. Always restart the runtime and reinstall TensorFlow-related packages to avoid lingering conflicts.

This ensures that all operations use the correct `tensorflow.keras` implementation, avoiding compatibility issues.


In [1]:
!pip uninstall -y keras tensorflow tensorflow-model-optimization
!pip install tensorflow==2.12 tensorflow-model-optimization

Found existing installation: keras 3.5.0
Uninstalling keras-3.5.0:
  Successfully uninstalled keras-3.5.0
Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00


In [1]:
# Import required libraries

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Part I:  Post Training Quantization (PTQ) - Integer, Dynamic Range, and Float 16
First part of this notebook demonstrates three types of post-training quantization for a CNN model trained on the MNIST dataset. Quantization is a model compression technique that reduces model size and computational requirements, enabling efficient deployment on resource-constrained devices.

## Goals:
1. Train a CNN model on the MNIST dataset.
2. Apply three quantization techniques:
   - Integer Quantization
   - Dynamic Range Quantization
   - Float 16 Quantization
3. Compare the quantized models in terms of size and accuracy.


## Dataset Preparation
We use the MNIST dataset, which contains grayscale images of handwritten digits (0-9).
1. Normalize the pixel values to the range [0, 1].
2. Reshape the data for input into the CNN model.
3. One-hot encode the labels for classification.


In [2]:
# Import required libraries
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Reshape the data for CNN input
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(f"Training data shape: {x_train.shape}, Labels shape: {y_train.shape}")
print(f"Test data shape: {x_test.shape}, Labels shape: {y_test.shape}")


11490434/11490434 [==============================] - 0s 0us/step
Training data shape: (60000, 28, 28, 1), Labels shape: (60000, 10)
Test data shape: (10000, 28, 28, 1), Labels shape: (10000, 10)


## Training a Simple CNN
We build a Convolutional Neural Network (CNN) with the following layers:
1. **Convolutional Layer**: Extracts features from the input images.
2. **MaxPooling Layer**: Reduces spatial dimensions, lowering computational requirements.
3. **Flatten Layer**: Converts the 2D feature maps into a 1D vector.
4. **Dense Layers**: Fully connected layers for classification.

The model is compiled using the Adam optimizer and trained for 2 epochs.


In [9]:
# Build a simple CNN model
def create_cnn_model():
    inputs = Input(shape=(28, 28, 1))
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(10, activation='softmax')(x)
    return Model(inputs, outputs)

# Compile and train the model
model = create_cnn_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=2, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the trained model
original_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"Original Model Accuracy: {original_accuracy:.4f}")


Epoch 1/2
1875/1875 [==============================] - 50s 26ms/step - loss: 0.1520 - accuracy: 0.9542 - val_loss: 0.0572 - val_accuracy: 0.9811
Epoch 2/2
1875/1875 [==============================] - 58s 31ms/step - loss: 0.0519 - accuracy: 0.9844 - val_loss: 0.0558 - val_accuracy: 0.9811
Original Model Accuracy: 0.9811


## Applying Quantization Techniques
We apply three types of post-training quantization to the trained CNN model:
1. **Integer Quantization**: Converts both weights and activations to 8-bit integers.
2. **Dynamic Range Quantization**: Reduces the precision of weights while keeping activations in floating-point format.
3. **Float 16 Quantization**: Converts both weights and activations to 16-bit floating-point values.


In [4]:
# Convert the trained model to TensorFlow Lite format without quantization (baseline)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the baseline TFLite model
with open("model_baseline.tflite", "wb") as f:
    f.write(tflite_model)

# Integer Quantization
def representative_data_gen():
    for input_value in x_test[:100]:  # Use a subset of the test set
        # Yield a dictionary where the key matches the model's input tensor name
        yield [input_value.reshape(1, 28, 28, 1).astype("float32")]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen  # Corrected function
converter.target_spec.supported_types = [tf.int8]
tflite_model_int8 = converter.convert()

# Save the Integer Quantization model
with open("model_integer_quant.tflite", "wb") as f:
    f.write(tflite_model_int8)

# Dynamic Range Quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_dynamic = converter.convert()

# Save the Dynamic Range Quantization model
with open("model_dynamic_quant.tflite", "wb") as f:
    f.write(tflite_model_dynamic)

# Float 16 Quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # Ensure fresh converter
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]  # Correctly set for Float16 quantization
tflite_model_float16 = converter.convert()

# Save the Float 16 Quantization model
with open("model_float16_quant.tflite", "wb") as f:
    f.write(tflite_model_float16)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


## Comparison of Model Sizes and Accuracy
We compare the quantized models in terms of:
1. **Model Size**: Smaller models are better suited for resource-constrained devices.
2. **Accuracy**: Quantization should preserve the original model’s accuracy as much as possible.


In [5]:
import os

# Compare model sizes
model_files = [
    "model_baseline.tflite",
    "model_integer_quant.tflite",
    "model_dynamic_quant.tflite",
    "model_float16_quant.tflite"
]

print("Model Sizes (KB):")
for file in model_files:
    print(f"{file}: {os.path.getsize(file) / 1024:.2f} KB")

# Evaluate accuracy for quantized models
def evaluate_tflite_model(tflite_model_path):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Evaluate on test set
    correct_predictions = 0
    for i in range(len(x_test)):
        input_data = x_test[i:i+1].astype("float32")
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        if tf.argmax(output_data, axis=1) == tf.argmax(y_test[i:i+1], axis=1):
            correct_predictions += 1

    return correct_predictions / len(x_test)

print("\nModel Accuracies:")
print(f"Baseline: {original_accuracy:.4f}")
print(f"Integer Quantization: {evaluate_tflite_model('model_integer_quant.tflite'):.4f}")
print(f"Dynamic Range Quantization: {evaluate_tflite_model('model_dynamic_quant.tflite'):.4f}")
print(f"Float 16 Quantization: {evaluate_tflite_model('model_float16_quant.tflite'):.4f}")


Model Sizes (KB):
model_baseline.tflite: 2713.34 KB
model_integer_quant.tflite: 682.25 KB
model_dynamic_quant.tflite: 682.25 KB
model_float16_quant.tflite: 1358.77 KB

Model Accuracies:
Baseline: 0.9843
Integer Quantization: 0.9842
Dynamic Range Quantization: 0.9842
Float 16 Quantization: 0.9843


# Part I Summary
Quantization significantly reduces model size while maintaining comparable accuracy. Key observations:
- Integer Quantization provides the smallest model size and computation complexity but may slightly reduce accuracy.
- Dynamic Range Quantization balances size, computation complexity, and performance.
- Float 16 Quantization retains higher accuracy with moderate size and computation complexity reduction.


## Part II: Quantization-Aware Training (QAT) - Integer and Dynamic Range
Quantization-Aware Training is a technique where integer quantization is simulated during the training process. This allows the model to adjust its weights and activations, minimizing the accuracy loss caused by quantization.

In this step, we:
1. Prepare a quantization-aware model using TensorFlow’s `QuantizeWrapper`.
2. Train the model on MNIST with quantization simulation.
3. Convert the model to TensorFlow Lite format for Integer and Dynamic Range.
4. Compare model sizes and accuracy after QAT.


In [7]:
# Verify the model type
model = create_cnn_model()
print(f"Model type: {type(model)}")

from tensorflow_model_optimization.quantization.keras import quantize_model

# Prepare the quantization-aware model
qat_model = quantize_model(create_cnn_model())

# Compile the QAT model
qat_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the QAT model
print("Training the QAT Model...")
qat_history = qat_model.fit(x_train, y_train, epochs=2, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the QAT-trained model
qat_accuracy = qat_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"QAT Model Accuracy: {qat_accuracy:.4f}")


Model type: <class 'keras.engine.functional.Functional'>
Training the QAT Model...
Epoch 1/3
1875/1875 [==============================] - 72s 37ms/step - loss: 0.1504 - accuracy: 0.9564 - val_loss: 0.0594 - val_accuracy: 0.9806
Epoch 2/3
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0527 - accuracy: 0.9842 - val_loss: 0.0497 - val_accuracy: 0.9838
Epoch 3/3
1875/1875 [==============================] - 71s 38ms/step - loss: 0.0332 - accuracy: 0.9899 - val_loss: 0.0474 - val_accuracy: 0.9849
QAT Model Accuracy: 0.9849


## Applying Quantization After QAT
We now convert the QAT-trained model to TensorFlow Lite format and apply three quantization techniques:
1. **Integer Quantization**
2. **Dynamic Range Quantization**


In [11]:
# Convert the QAT-trained model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)

# Integer Quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen  # Use the same representative data generator
converter.target_spec.supported_types = [tf.int8]
tflite_model_qat_int8 = converter.convert()

# Save Integer Quantization model
with open("qat_model_integer_quant.tflite", "wb") as f:
    f.write(tflite_model_qat_int8)

# Dynamic Range Quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_qat_dynamic = converter.convert()

# Save Dynamic Range Quantization model
with open("qat_model_dynamic_quant.tflite", "wb") as f:
    f.write(tflite_model_qat_dynamic)


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


## Comparison of Model Sizes and Accuracy
We compare the model sizes and accuracy for the quantized QAT-trained models.


In [12]:
# Compare model sizes
model_files = [
    "qat_model_integer_quant.tflite",
    "qat_model_dynamic_quant.tflite",
]

print("\nModel Sizes After QAT (KB):")
for file in model_files:
    print(f"{file}: {os.path.getsize(file) / 1024:.2f} KB")

# Evaluate accuracy for QAT-quantized models
print("\nModel Accuracies After QAT:")
print(f"QAT Baseline Model Accuracy: {qat_accuracy:.4f}")
print(f"Integer Quantization: {evaluate_tflite_model('qat_model_integer_quant.tflite'):.4f}")
print(f"Dynamic Range Quantization: {evaluate_tflite_model('qat_model_dynamic_quant.tflite'):.4f}")



Model Sizes After QAT (KB):
qat_model_integer_quant.tflite: 682.84 KB
qat_model_dynamic_quant.tflite: 682.84 KB

Model Accuracies After QAT:
QAT Baseline Model Accuracy: 0.9849
Integer Quantization: 0.9849
Dynamic Range Quantization: 0.9849


# Part II Summary
Quantization-Aware Training (QAT) significantly reduces accuracy loss caused by quantization. Key observations:
- QAT improves the accuracy of quantized models, making it suitable for resource-constrained deployments that demands the high precision/accuracy.
- QAT is incompatible with Float16 quantization because they target fundamentally different quantization formats.
